# Reading the train data from file and training the model

In [17]:
''' Reading the train.txt file into raw_data '''

raw_data = []                                              # Can be used for further processing
unigram = {}
error_dictionary = {}
sentences_list = []
filename = "/home/ashwini/Dropbox/Sem3/NLP/P1B/Data/train.txt"
file = open(filename, "r")
for line in file.readlines():
    line = line.split('\n')
    raw_data.append(line[0])
file.close()

# Dictionary to maintain the mapping of errors
map_errtoint = {'Nn':1,'Cit':2,'Rloc-':3,'Mec':4,'ArtOrDet':5,'Vform':6,'V0':7,'SVA':8,'Others':9,'Vt':10,'Wform':11,'Pform':12,'Wci':13,'Trans':14,'Sfrag':15,'Um':16,'Prep':17,'Pref':18,'Spar':19,'WOinc':20,'Wtone':21,'Vm':22,'WOadv':23,'Srun':24,'Npos':25,'Wa':26,'Smod':27,'Ssub':28,'Noerror':29}
map_inttoerr = {value: key for key, value in map_errtoint.items()}

In [18]:
import numpy as np
incorrect_sentences_list = []
correct_sentences_list = []

str_correct = ""
str_incorrect = ""
str_error = ""

error_sentences_list = []
l = []

trigram_freq = {}
correction = {}
process_data = []

start_of_sent = "<S>"
end_of_sent = "<E>"
process_data.append(start_of_sent)

for i in range(0,len(raw_data)):
    if raw_data[i] == '':
        continue
    elif raw_data[i]==".":
        process_data.append(end_of_sent)
        process_data.append(start_of_sent)
        i +=1
    else:
        process_data.append(raw_data[i]) 



# Creating the Main Dictionary

In [19]:
'''
Creating the main data structure : Dictionary which will store the errorneous trigram along with its 
frequency and corrected triagram for each Error class 
'''
# Creating triagrams from the raw data given to us
pre2 = process_data[1]
pre1 = process_data[2]
i = 3

while i<len(process_data):
    curr = process_data[i]
    
    if curr == "<E>":                   # Skip the end of the sentence tag from getting into the triagram
        pre2 = process_data[i+2]
        pre1 = process_data[i+3]
        i += 4
        continue
    
    corr_tri = []
    incorr_tri = []
    error = []
    word_error = []
    
    if(pre2.count('   ') == 2):
        temp = pre2.split('   ')
        incorr_tri.append(str(temp[0]))
        corr_tri.append(temp[1])
        error.append(temp[2])
        word_error.append(temp[2])
    else:
        incorr_tri.append(pre2)
        corr_tri.append(pre2)
        word_error.append('')
        
    
    if(pre1.count('   ') == 2):
        temp = pre1.split('   ')
        incorr_tri.append(str(temp[0]))
        
        if temp[1] in corr_tri:
            corr_tri.append('')
        else:
            corr_tri.append(temp[1])
        
        error.append(temp[2])
        word_error.append(temp[2])
    else:
        incorr_tri.append(pre1)
        corr_tri.append(pre1)
        word_error.append('')
        
    
    if(curr.count('   ') == 2):
        temp = curr.split('   ')
        incorr_tri.append(str(temp[0]))
        
        if temp[1] in corr_tri:
            corr_tri.append('')
        else:
            corr_tri.append(temp[1])
        
        error.append(temp[2])
        word_error.append(temp[2])
    else:
        incorr_tri.append(curr)
        corr_tri.append(curr)
        word_error.append('')
    
    pre2 = pre1
    pre1 = curr
    i += 1
    
    error = set(error)
    tup_incorr = tuple(incorr_tri)
    # print(tup_incorr)
    tup_corr = tuple(corr_tri)
    # print(tup_corr)
    tup_word_error = tuple(word_error)
    # print(tup_word_error)
    
    if(len(error)==0):
        num = 29
        if num not in trigram_freq:
            temp_dict ={}
            temp_dict[tup_incorr] = (1,tup_corr,tup_word_error)
            trigram_freq[num] = temp_dict

        else:
            temp_dict = {}
            temp_dict = trigram_freq[num]
            if tup_incorr not in temp_dict:
                trigram_freq[num].update({tup_incorr:(1,tup_corr,tup_word_error)})

            else:
                val = temp_dict[tup_incorr]
                v = val[0] + 1
                temp_dict[tup_incorr] = (v,tup_corr,tup_word_error)
                
    else:
        for j in error:
            num = map_errtoint.get(j,-1)
            
            if num not in trigram_freq:
                # print(num)
                temp_dict ={}
                temp_dict[tup_incorr] = (1,tup_corr,tup_word_error)
                trigram_freq[num] = temp_dict

            else:
                temp_dict = {}
                temp_dict = trigram_freq[num]
                if tup_incorr not in temp_dict:
                    trigram_freq[num].update({tup_incorr:(1,tup_corr,tup_word_error)})

                else:
                    val = temp_dict[tup_incorr]
                    v = val[0] + 1
                    temp_dict[tup_incorr] = (v,tup_corr,tup_word_error)



In [20]:
# Maintaining count of triagrams in each error class and the number of total triagrams seen in the complete dataset   

size_dict = {}
Total_trigrams = 0
for key,value in trigram_freq.items():
    for k,v in trigram_freq[key].items():
        size_dict[key] = size_dict.get(key,0) + v[0]
    Total_trigrams += size_dict[key]

# Smoothing

In [21]:
''' Good Turing for Smoothing : Handling unseen triagrams ''' 

# Invert the dictionary to maintain freq-of-freq 
def eq_classes(tm):
    eq = {}
    for k, v in tm.items():
        if v not in eq:
            eq[v[0]] = []
        eq[v[0]].append(k)
    return eq

def good_turing_smoothing(tm,N):
    Nr = eq_classes(tm)
    
    nr_counts = {k : len(v) for k, v in Nr.items()}
    nr_probs = {k : (k*v)/float(N) for k, v in nr_counts.items()}
    
    sorted_nrs = sorted(nr_counts.items())
    sorted_probs = sorted(nr_probs.items())
    MAX = sorted_nrs[0][1]
    
    new_nrs = {}
    for r, nr in Nr.items():
        if (r+1) in Nr:
            new_nr = ((r+1) * len(Nr[r+1])) / float(N) 
        else:
            new_nr = MAX*r**-2 / float(N)
        new_nrs[r] = new_nr
        
    return new_nrs[1]

# Detecting error for a single sentence

In [22]:
# sent = "the life expectancy of individuals has increased due to better accessibility to healthcare services and facilities than before"
# #sent = "governments all around the world has to make provision for improved healthcare as - live expectancy of their citizens and medical technological advancement are improving by the years."
# sent = sent.split(" ")
# pre2 = sent[0]
# pre1 = sent[1]

# for i in range(2,len(sent)):
#     curr = sent[i]
#     trigram = (pre2,pre1,curr)
#     error_class = -1
#     prob = -10.00 
    
#     for num in range(1,30):
#         temp_dict = {}
#         temp_dict = trigram_freq[num]
#         val = temp_dict.get(trigram,())
        
#         if len(val) != 0:
#             v = float(val[0])/ float(size_dict[num])
#             print(str(trigram) + "   " + map_inttoerr[num] + "    " + str(v)),
#             if prob<v:
#                 error_class = num
#                 prob = v
#         else:
#             v = good_turing_smoothing(trigram_freq[num],Total_trigrams)
#             if prob<v:
#                 error_class = num
#                 prob = v
    
    
#     if(error_class != 29 and error_class != -1):
#         print(error_class)
#         print(str(trigram) + "   " + map_inttoerr[error_class] + "    " + str(prob))

#     pre2 = pre1
#     pre1 = curr

# Testing on given test file

In [23]:
''' Read Test File '''

filepath1 = "/home/ashwini/Dropbox/Sem3/NLP/P1B/Data/dev.txt"
file1 = open(filepath1,'r')

test_data = []

for lines in file1.readlines():
    lines = lines.split('\n')
    test_data.append(lines[0])
    
file1.close()
    

# Processing the given test file

In [24]:
''' Generate trigrams for test data '''

trigram_freq_testdata = []
process_testdata = []

start_of_sent = "<S>"
end_of_sent = "<E>"
process_testdata.append(start_of_sent)

for i in range(0,len(test_data)):
    # print(i)
    if test_data[i] == '':
        continue
    elif test_data[i]==".":
        process_testdata.append(end_of_sent)
        process_testdata.append(start_of_sent)
        i +=1
    else:
        ss = test_data[i].split("   ")
        process_testdata.append(ss[0])

# Creating triagram for the test data and Naive Bayes

In [25]:
pre2 = process_testdata[1]
pre1 = process_testdata[2]
i = 3
output = []
output1 = []
while i in range(0,len(process_testdata)):
    curr = process_testdata[i]
    if curr == "<E>":
        if i+2 < len(process_testdata):
            pre2 = process_testdata[i+2]
            pre1 = process_testdata[i+3]
            i += 4
            continue
        else:
            break
        
    trigram = (pre2,pre1,curr)
#     print(trigram)
    trigram_freq_testdata.append(trigram)
    
    pre2 = pre1
    pre1 = curr
    i += 1
    
    error_class = -1
    prob = -10.00 
    incorr_word = []
    corr_word = []
    
    for num in range(1,30):
        temp_dict = {}
        temp_dict = trigram_freq[num]
        val = temp_dict.get(trigram,())
        
        if len(val) != 0:
            # Naive Bayes formula implemented here
            v = float(val[0])/ float(size_dict[num])
            v *= float(size_dict[num]) / float(Total_trigrams)
            # As values were too small so every value is multiplied by same number
            v *= 100000
            if prob<v:
                error_class = num
                prob = v
                
                incorr_word.clear()
                corr_word.clear()
                
                temp_list = val[2]
                corr_word = list(val[1])
                
                for it in range(0,len(temp_list)):
                    if temp_list[it] == map_inttoerr[num]:
                        incorr_word.append(trigram[it])
                    
#If the trigram is not present in seen triagrams of dataset then smoothing is applied to handle such situation                       
        else:
#             v = good_turing_smoothing(trigram_freq[num],Total_trigrams)
#             if prob<v:
#                 error_class = num
#                 prob = v
            pass

    # Identifying the words with error in triagram and then finding the corresponding error and correction
    sss = ""
    sss1 = ""
    if(error_class != -1 and error_class != 29):
        for t in trigram:
            sss = ""
            sss1 = ""
            if t in incorr_word:
                index = trigram.index(t)
                correction = corr_word[index]
                sss = t + "   " + correction +"   " + map_inttoerr[error_class]
                sss1 = t + "   " + map_inttoerr[error_class]
#                 print(sss)
            else:
                sss = t
                sss1 = t
                 
            if len(output)>2:
                if len(sss.split("   ")) == 3 and sss.split("   ")[0] == output[-1]:
                    output[-1] = sss
                    output1[-1] = sss1
                    continue
                if len(sss.split("   ")) == 3 and sss.split("   ")[0] == output[-2]:
                    output[-2] = sss
                    output1[-2] = sss1
                    continue
                elif sss != output[-1] and sss != output[-2]:
                    output.append(sss)
                    output1.append(sss1)
            else:
                output.append(sss)
                output1.append(sss1)
            
    # Handling repeated entries while inserting in output file dev-results    
    else:
        for t in trigram:
            if len(output)>=3:
                # print(output[-1] + " " + output[-2])
                if (len(output[-1].split("   ")) == 1 and t == output[-1]) or (len(output[-2].split("   ")) == 1 and t == output[-2]):
                    continue
                if (len(output[-1].split("   ")) == 3 and t == output[-1].split("   ")[0]) or (len(output[-2].split("   ")) == 3 and t == output[-2].split("   ")[0]):
                    continue
                if (len(output[-3].split("   ")) == 3 and t == output[-3].split("   ")[0]) or (len(output[-3].split("   ")) == 1 and t == output[-3]):
                    continue
                else:                
                    output.append(t)
                    output1.append(t)
            else:
                output.append(t)
                output1.append(t)


In [26]:
'''
Writing in output file
'''
file_output = open("/home/ashwini/Dropbox/Sem3/NLP/P1B/Data/dev_correction_result.txt","w+")
for ii in output:
#     print(ii)
    file_output.write(ii+"\n")  

file_output.close()

# file_output = open("/home/ashwini/Dropbox/Sem 3/NLP/P1B/Data/dev_result.txt","w+")
# for ii in output1:
# #     print(ii)
#     file_output.write(ii+"\n")  

# file_output.close()